<b>Importing libraries and scraping wikipedia for postalcode table<b>

In [7]:
import requests 
from bs4 import BeautifulSoup 
import pandas as pd

In [8]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
resp=requests.get(url) 
column_names = ['Postalcode','Borough','Neighborhood']

#http_respone 200 means OK status 
if resp.status_code==200: 
    print("Successfully opened the web page") 
    
    # we need a parser,Python built-in HTML parser is enough . 
    soup=BeautifulSoup(resp.text,'html.parser')     
  
    # table is the list which contains all the text 
    table=soup.find("table",{"class":"wikitable sortable"}) 

else: 
    print("Error")

Successfully opened the web page


<b>Create dataframe based in the scrape table, removing records with Borough that are "Not assigned" and assigning Borough value to neighbourhood that are "Not assigned".<b>

In [9]:
df_table = pd.read_html(str(table))
df_table = df_table[0].dropna(axis=0)
df_table.columns = column_names
df_table.drop(df_table.loc[df_table['Borough']=='Not assigned'].index, inplace=True)
i = 0
for i in range(0,df_table.shape[0]):
    if df_table.iloc[i][2] == 'Not assigned':
        df_table.iloc[i][2] = df_table.iloc[i][1]
        i = i+1
df_table.reset_index(drop=True,inplace=True)


<b>Combine rows with common postal code and separating by comma<b>

In [10]:
df_table = df_table.groupby(['Postalcode','Borough'])['Neighborhood'].apply(', '.join).reset_index()
df_table

,Postalcode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ..."
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest..."



<b>Show final results<b>

In [11]:
df_table.shape

(103, 3)

<b> Working with the given coordinates <b>

In [21]:
df_coord = pd.read_csv('C:/Users/RohitSingh/Desktop/DATA_SCIENCE/Geospatial_Coordinates.csv')
df_coord

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


<b> Import the datasets with the coordinates and then merge with the dataframe <b>

In [19]:
df_total = df_table.merge(df_coord, how='inner', left_on='Postalcode', right_on='Postal Code')

In [20]:
df_total.drop(['Postal Code'], axis=1, inplace=True)
df_total

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ...",43.688905,-79.554724
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",43.739416,-79.588437
